In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import google.generativeai as genai
import os
from dotenv import load_dotenv

In [2]:
with open("newDataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

contexts = [entry["context"] for entry in data]
questions = [entry["question"] for entry in data]

In [3]:
# Step 1: Embed contexts
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = model.encode(contexts, convert_to_numpy=True)

In [4]:
# Step 2: Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [5]:
# Save for later use
faiss.write_index(index, "context.index")
np.save("contexts.npy", np.array(contexts))

In [6]:
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
llm = genai.GenerativeModel("gemini-2.0-flash")

# Load index and data
index = faiss.read_index("context.index")
contexts = np.load("contexts.npy", allow_pickle=True)
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def generate_answer(query: str, top_k=1):
    query_vec = embed_model.encode([query])
    D, I = index.search(query_vec, top_k)
    retrieved_context = "\n".join([contexts[i] for i in I[0]])

    prompt = f"""
Câu hỏi: {query}
Nội dung nội quy liên quan:
{retrieved_context}

Trả lời ngắn gọn, chính xác dựa trên nội dung trên:
"""
    response = llm.generate_content(prompt)
    return response.text.strip()

In [8]:
print(generate_answer("Sinh viên có được hút thuốc hoặc uống rượu bia trong trường không?"))

Không, sinh viên không được hút thuốc hoặc uống rượu bia trong trường.
